In [76]:
import pandas as pd
import numpy as np
import autoviz
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn import svm, datasets, metrics, neighbors
from sklearn.ensemble import RandomForestRegressor
import math
from sklearn.model_selection import GridSearchCV, cross_validate
import random

## **Preprocessing**

In [77]:
dataframe_train = pd.read_csv("C:/Users/henim/Minor Project/train.csv");

In [78]:
dataframe_train.head()

Gender  Age  openness  neuroticism  conscientiousness  agreeableness  \
0    Male  17       7          4               7                3         
1    Male  19       4          5               4                6         
2  Female  18       7          6               4                5         
3  Female  22       5          6               7                4         
4  Female  19       7          4               6                5         

   extraversion Personality (Class label)  
0        2              extraverted        
1        6                  serious        
2        5               dependable        
3        3              extraverted        
4        4                   lively

In [79]:
dataframe_test = pd.read_csv("C:/Users/henim/Minor Project/test.csv")
dataframe_test.head()

Gender  Age  openness  neuroticism  conscientiousness  agreeableness  \
0  Female  20       7          9               9                5         
1    Male  17       5          4               5                2         
2  Female  25       5          5               7                2         
3  Female  18       6          2               7                4         
4  Female  19       2          4               7                1         

   extraversion Personality (class label)  
0        5               dependable        
1        4                  serious        
2        4                  serious        
3        7                  serious        
4        3              responsible

In [80]:
train_length = len(dataframe_train)
print(f"Train Length: {train_length}")
test_length = len(dataframe_test)
print(f"Test Length: {test_length}")

Train Length: 709
Test Length: 315


In [81]:
dataframe_train.dtypes

Gender                       object
Age                           int64
openness                      int64
neuroticism                   int64
conscientiousness             int64
agreeableness                 int64
extraversion                  int64
Personality (Class label)    object
dtype: object

In [82]:
dataframe_test.dtypes
dataframe_train.rename(columns={'Personality (Class label)':'Personality'}, inplace=True)
dataframe_test.rename(columns={'Personality (class label)':'Personality'}, inplace=True)

In [83]:
dataframe_train['Personality'].value_counts()

serious        161
extraverted    150
dependable     138
lively         134
responsible    126
Name: Personality, dtype: int64

In [84]:
dataframe_test['Personality'].value_counts()

serious        153
extraverted     77
responsible     40
lively          24
dependable      21
Name: Personality, dtype: int64

In [85]:
dataframe_test.head()

Gender  Age  openness  neuroticism  conscientiousness  agreeableness  \
0  Female  20       7          9               9                5         
1    Male  17       5          4               5                2         
2  Female  25       5          5               7                2         
3  Female  18       6          2               7                4         
4  Female  19       2          4               7                1         

   extraversion Personality   
0        5        dependable  
1        4           serious  
2        4           serious  
3        7           serious  
4        3       responsible

In [86]:
combinedDataframe = pd.concat([dataframe_train,dataframe_test])
print(f"New Length: {len(combinedDataframe)}")
combinedDataframe['Personality'].value_counts()

New Length: 1024


serious        314
extraverted    227
responsible    166
dependable     159
lively         158
Name: Personality, dtype: int64

In [87]:
shuffled_df = combinedDataframe.sample(frac=1).reset_index(drop=True)
shuffled_df.head()

Gender  Age  openness  neuroticism  conscientiousness  agreeableness  \
0    Male  17       6          6               7                5         
1    Male  24       5          6               7                4         
2    Male  20       7          5               4                1         
3  Female  21       5          6               7                4         
4  Female  20       3          7               3                2         

   extraversion Personality   
0        7           serious  
1        5           serious  
2        5        dependable  
3        6        dependable  
4        5       extraverted

In [88]:
label = LabelEncoder()
combinedDataframe['Gender'] = label.fit_transform(combinedDataframe['Gender'])
combinedDataframe['Personality'] = label.fit_transform(combinedDataframe['Personality'])
print(label.classes_)

['dependable' 'extraverted' 'lively' 'responsible' 'serious']


In [89]:
combinedDataframe

Gender  Age  openness  neuroticism  conscientiousness  agreeableness  \
0       2    17       7          4               7                3         
1       2    19       4          5               4                6         
2       1    18       7          6               4                5         
3       1    22       5          6               7                4         
4       1    19       7          4               6                5         
..      ...  ...       ...          ...                ...            ...   
310     1    19       6          5               6                4         
311     2    18       2          5               8                3         
312     2    18       7          5               6                2         
313     2    23       6          7               5                4         
314     1    18       5          7               3                5         

     extraversion  Personality  
0          2            1       
1          6            4       
2          5            0       
3          3            1       
4          4            2       
..            ...          ...  
310        3            1       
311        7            0       
312        7            4       
313        3            1       
314        6            1       

[1024 rows x 8 columns]

In [90]:
# scaler = StandardScaler()
# combinedDataframe[input_columns] = scaler.fit_transform(combinedDataframe[input_columns])

In [91]:
input_columns = ['Gender', 'Age', 'openness', 'neuroticism','conscientiousness', 'agreeableness','extraversion']
output_columns = ['Personality']

In [92]:
combinedDataframe

Gender  Age  openness  neuroticism  conscientiousness  agreeableness  \
0       2    17       7          4               7                3         
1       2    19       4          5               4                6         
2       1    18       7          6               4                5         
3       1    22       5          6               7                4         
4       1    19       7          4               6                5         
..      ...  ...       ...          ...                ...            ...   
310     1    19       6          5               6                4         
311     2    18       2          5               8                3         
312     2    18       7          5               6                2         
313     2    23       6          7               5                4         
314     1    18       5          7               3                5         

     extraversion  Personality  
0          2            1       
1          6            4       
2          5            0       
3          3            1       
4          4            2       
..            ...          ...  
310        3            1       
311        7            0       
312        7            4       
313        3            1       
314        6            1       

[1024 rows x 8 columns]

In [93]:
# No null values
combinedDataframe.isna().sum()

Gender               0
Age                  0
openness             0
neuroticism          0
conscientiousness    0
agreeableness        0
extraversion         0
Personality          0
dtype: int64

In [94]:
# Generating test and train datasets
train_length = (int)(0.8*train_length)
dataframe_train = combinedDataframe[:train_length]
dataframe_test = combinedDataframe[train_length:]

In [95]:
X = np.array(dataframe_train[input_columns])
Y = np.array(dataframe_train[output_columns])

In [96]:
X.shape

(567, 7)

In [97]:
Y.shape

(567, 1)

In [98]:
X_test = np.array(dataframe_test[input_columns])
Y_test = np.array(dataframe_test[output_columns])

In [55]:
def categorical_histogram(df, colname, figscale=1, mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  df.groupby(colname).size().plot(kind='barh', color=sns.palettes.mpl_palette(mpl_palette_name), figsize=(8*figscale, 4.8*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  return autoviz.MplChart.from_current_mpl_state()

chart = categorical_histogram(dataframe_train, *['Personality'], **{})
chart

AttributeError: module 'autoviz' has no attribute 'MplChart'

# **K - Neighbours Classification**

In [99]:
clf = neighbors.KNeighborsClassifier()
param_grid =[{'n_neighbors':[200,539,1023,1211,45,50,100,1535,2311,150,300,25,44,2637],'weights':['uniform', 'distance']}]

In [100]:
gridSearch = GridSearchCV(clf, param_grid, cv=100)
gridSearch.fit(X,Y)

GridSearchCV(cv=100, estimator=KNeighborsClassifier(),
             param_grid=[{'n_neighbors': [200, 539, 1023, 1211, 45, 50, 100,
                                          1535, 2311, 150, 300, 25, 44, 2637],
                          'weights': ['uniform', 'distance']}])

In [101]:
knnPrediction = gridSearch.predict(X_test)
print(knnPrediction)

[2 4 4 4 3 4 4 1 4 0 4 1 0 0 4 1 4 4 4 0 2 4 4 4 2 4 4 4 1 4 4 4 4 0 0 4 1
 4 1 1 1 0 4 4 1 1 1 1 4 4 1 2 4 1 4 4 4 4 4 1 4 4 1 4 4 4 4 4 1 2 1 1 1 4
 3 3 1 4 1 2 1 4 4 2 4 0 2 2 4 4 1 3 4 4 4 1 4 3 4 4 1 3 3 1 2 1 1 1 4 4 1
 1 4 1 4 2 1 3 4 4 1 3 1 4 4 0 4 3 4 4 1 4 4 4 4 1 2 2 4 3 1 2 4 3 4 4 4 2
 4 4 4 4 4 3 1 4 4 4 2 4 4 4 4 1 4 4 4 0 1 4 1 4 2 4 4 4 4 4 4 3 4 4 2 1 4
 4 4 4 4 4 1 4 4 4 4 1 0 4 1 4 2 4 4 1 4 4 0 4 3 4 4 3 4 4 1 4 4 4 0 4 1 4
 4 4 4 1 2 2 4 4 4 2 4 3 4 2 4 4 4 1 2 4 4 4 1 1 4 4 4 4 4 4 4 1 4 4 1 4 3
 3 3 1 2 4 4 4 4 4 4 4 3 0 4 2 4 0 0 4 1 4 3 0 4 4 4 4 4 4 1 2 4 4 4 4 1 0
 4 4 4 4 4 3 1 4 4 0 4 3 1 4 4 4 1 1 4 1 4 3 4 4 4 4 3 4 4 4 4 4 3 4 4 4 4
 1 1 2 1 1 4 0 1 2 4 4 1 2 4 4 2 4 4 4 1 4 4 4 1 4 4 0 0 4 4 4 0 4 4 0 1 4
 2 1 1 4 1 4 4 1 4 4 1 1 2 4 1 4 1 1 4 4 4 4 0 4 4 4 2 4 3 0 2 2 3 1 2 4 4
 4 4 4 4 4 0 4 4 4 4 4 4 1 4 4 4 4 3 4 4 0 2 2 4 4 1 4 4 4 1 4 4 2 4 1 4 4
 4 4 4 2 2 3 4 4 4 4 4 4 4]


In [102]:
print(f"Accuracy: {(metrics.accuracy_score(Y_test, knnPrediction)*100).round(2)}%")
print("Confusion Matrix: ")
print(metrics.confusion_matrix(Y_test, knnPrediction))
print("Classification Report: ")
print(metrics.classification_report(Y_test,knnPrediction))

Accuracy: 38.51%
Confusion Matrix: 
[[  5   7   2   2  35]
 [  7  33   5   1  56]
 [  3   9  11   5  27]
 [  3  13   3  10  33]
 [ 10  26  21  13 117]]
Classification Report: 
              precision    recall  f1-score   support

           0       0.18      0.10      0.13        51
           1       0.38      0.32      0.35       102
           2       0.26      0.20      0.23        55
           3       0.32      0.16      0.22        62
           4       0.44      0.63      0.51       187

    accuracy                           0.39       457
   macro avg       0.31      0.28      0.29       457
weighted avg       0.36      0.39      0.36       457



## **Gaussian Naive Bayes**

In [103]:
clf = GaussianNB()
clf.fit(X,Y, sample_weight=4)
Predicted_class = clf.predict(X_test)

In [104]:
Predicted_class

array([2, 1, 4, 1, 2, 4, 4, 1, 1, 4, 1, 1, 0, 4, 1, 1, 2, 4, 1, 1, 0, 1,
       4, 4, 3, 4, 2, 1, 1, 4, 2, 4, 1, 4, 1, 4, 2, 4, 3, 1, 4, 0, 1, 4,
       2, 1, 1, 1, 4, 1, 1, 0, 4, 1, 0, 4, 0, 1, 4, 0, 4, 0, 4, 1, 4, 4,
       2, 4, 4, 2, 1, 3, 1, 2, 3, 4, 4, 4, 1, 4, 1, 0, 4, 4, 4, 1, 0, 0,
       4, 0, 0, 3, 0, 4, 4, 0, 4, 3, 1, 4, 4, 3, 4, 4, 2, 3, 1, 1, 4, 0,
       0, 1, 4, 1, 4, 2, 1, 3, 4, 4, 1, 4, 1, 4, 4, 1, 4, 2, 4, 3, 1, 1,
       1, 1, 1, 4, 4, 2, 4, 2, 4, 4, 1, 4, 1, 4, 0, 4, 0, 1, 4, 4, 4, 4,
       2, 4, 4, 1, 4, 4, 4, 1, 4, 1, 4, 1, 4, 1, 4, 1, 1, 3, 2, 1, 1, 1,
       4, 4, 4, 4, 1, 0, 2, 0, 0, 0, 1, 1, 1, 4, 1, 1, 4, 2, 0, 1, 4, 1,
       0, 4, 4, 4, 3, 1, 0, 4, 0, 4, 0, 4, 4, 4, 4, 1, 4, 3, 3, 4, 4, 2,
       2, 1, 4, 4, 4, 1, 0, 4, 4, 4, 4, 3, 4, 2, 4, 0, 4, 1, 4, 1, 0, 4,
       1, 1, 1, 2, 4, 1, 4, 4, 4, 4, 4, 0, 4, 4, 1, 1, 3, 4, 3, 1, 2, 3,
       4, 0, 2, 4, 4, 4, 3, 4, 4, 0, 3, 4, 4, 4, 4, 1, 4, 4, 4, 4, 4, 4,
       1, 4, 4, 4, 1, 1, 4, 4, 1, 4, 4, 4, 1, 4, 1,

In [105]:
print("Accuracy: ",(metrics.accuracy_score(Y_test, Predicted_class)*100).round(2))
print("Confusion Matrix: ")
print(metrics.confusion_matrix(Y_test, Predicted_class))
print("Classification Report: ")
print(metrics.classification_report(Y_test,Predicted_class))

Accuracy:  41.58
Confusion Matrix: 
[[  9  12   0   0  30]
 [ 10  55   5   2  30]
 [  2  14   9   6  24]
 [  5  15   9  11  22]
 [ 26  31  13  11 106]]
Classification Report: 
              precision    recall  f1-score   support

           0       0.17      0.18      0.17        51
           1       0.43      0.54      0.48       102
           2       0.25      0.16      0.20        55
           3       0.37      0.18      0.24        62
           4       0.50      0.57      0.53       187

    accuracy                           0.42       457
   macro avg       0.34      0.32      0.32       457
weighted avg       0.40      0.42      0.40       457



# **Multinomial Naive Bayes Classification**

In [106]:
model = LogisticRegression(multi_class='multinomial',solver='newton-cg',max_iter=5000, C=1.0)
model.fit(X,Y)

LogisticRegression(max_iter=5000, multi_class='multinomial', solver='newton-cg')

In [107]:
Predicted_class = model.predict(X_test)
Predicted_class

array([2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 4, 4, 1, 0, 4,
       4, 4, 3, 4, 2, 1, 1, 1, 1, 0, 1, 1, 1, 4, 1, 1, 3, 1, 1, 4, 1, 4,
       1, 1, 1, 4, 1, 1, 1, 0, 4, 1, 2, 1, 4, 1, 4, 4, 4, 4, 1, 0, 4, 1,
       2, 1, 4, 3, 2, 1, 1, 1, 3, 3, 1, 1, 1, 0, 1, 4, 4, 2, 4, 1, 0, 0,
       4, 4, 4, 3, 0, 4, 4, 0, 4, 3, 1, 4, 4, 3, 4, 1, 1, 3, 1, 1, 4, 0,
       0, 1, 1, 4, 1, 3, 1, 3, 4, 1, 1, 4, 1, 4, 1, 1, 4, 2, 1, 4, 1, 1,
       4, 1, 1, 0, 4, 1, 4, 2, 1, 0, 0, 4, 0, 4, 3, 0, 4, 4, 4, 4, 4, 3,
       1, 4, 4, 1, 4, 4, 4, 1, 4, 1, 0, 2, 4, 1, 1, 1, 1, 3, 3, 2, 1, 2,
       4, 4, 4, 3, 1, 4, 2, 1, 4, 4, 4, 4, 1, 0, 4, 1, 4, 2, 0, 2, 1, 4,
       1, 1, 4, 4, 3, 2, 4, 4, 1, 4, 3, 4, 4, 4, 4, 4, 4, 4, 3, 1, 4, 4,
       0, 1, 4, 4, 4, 1, 4, 2, 4, 4, 4, 3, 4, 3, 4, 0, 4, 1, 4, 1, 0, 4,
       1, 1, 1, 1, 1, 4, 4, 1, 4, 4, 4, 0, 2, 4, 4, 1, 3, 3, 4, 1, 2, 2,
       3, 4, 2, 4, 4, 4, 3, 4, 4, 0, 4, 1, 1, 4, 4, 1, 4, 4, 0, 4, 4, 4,
       4, 1, 4, 4, 4, 1, 4, 2, 1, 1, 4, 3, 1, 4, 4,

In [108]:
print("Accuracy: ",(metrics.accuracy_score(Y_test, Predicted_class)*100).round(2))
print("Confusion Matrix: ")
print(metrics.confusion_matrix(Y_test, Predicted_class))
print("Classification Report: ")
print(metrics.classification_report(Y_test,Predicted_class))

Accuracy:  52.3
Confusion Matrix: 
[[  7  19   1   1  23]
 [  4  79   2   1  16]
 [  4  18  15   6  12]
 [  2  23   3  20  14]
 [ 21  30   6  12 118]]
Classification Report: 
              precision    recall  f1-score   support

           0       0.18      0.14      0.16        51
           1       0.47      0.77      0.58       102
           2       0.56      0.27      0.37        55
           3       0.50      0.32      0.39        62
           4       0.64      0.63      0.64       187

    accuracy                           0.52       457
   macro avg       0.47      0.43      0.43       457
weighted avg       0.52      0.52      0.51       457



# **Decision Tree**

In [109]:
clf = DecisionTreeClassifier(criterion='gini',max_depth=50,splitter='best')
clf.fit(X,Y)
Predicted_class = clf.predict(X_test)

In [110]:
Predicted_class

array([2, 1, 4, 4, 2, 3, 4, 2, 1, 2, 4, 1, 1, 0, 0, 0, 1, 2, 3, 0, 0, 4,
       4, 4, 1, 3, 4, 4, 0, 1, 4, 0, 1, 3, 0, 4, 1, 4, 4, 2, 1, 4, 0, 2,
       3, 1, 2, 0, 0, 3, 2, 0, 4, 1, 0, 0, 0, 1, 2, 2, 4, 0, 1, 1, 0, 0,
       4, 0, 2, 2, 1, 4, 1, 2, 0, 0, 1, 4, 1, 2, 0, 4, 1, 4, 0, 1, 1, 4,
       3, 0, 1, 2, 4, 3, 4, 1, 0, 3, 2, 4, 2, 0, 3, 2, 3, 4, 1, 1, 4, 1,
       0, 0, 0, 2, 3, 2, 4, 3, 3, 0, 1, 0, 1, 4, 3, 1, 3, 3, 0, 3, 1, 2,
       2, 3, 2, 2, 4, 4, 1, 4, 1, 1, 1, 2, 3, 0, 4, 0, 1, 0, 4, 4, 0, 3,
       4, 3, 3, 1, 2, 4, 0, 3, 0, 1, 0, 0, 3, 1, 4, 3, 1, 3, 2, 0, 2, 3,
       4, 1, 4, 1, 3, 1, 2, 4, 3, 2, 0, 3, 1, 1, 3, 1, 2, 4, 1, 4, 0, 0,
       1, 2, 3, 4, 2, 4, 0, 3, 1, 4, 3, 4, 1, 1, 4, 4, 4, 3, 2, 1, 4, 0,
       0, 4, 4, 4, 0, 1, 3, 0, 1, 0, 4, 2, 4, 3, 4, 3, 4, 2, 0, 0, 1, 4,
       1, 3, 2, 1, 4, 0, 1, 4, 0, 2, 4, 2, 4, 1, 3, 1, 3, 4, 3, 0, 1, 2,
       4, 0, 4, 1, 2, 1, 1, 4, 1, 0, 0, 3, 1, 3, 1, 1, 1, 0, 4, 3, 0, 3,
       0, 0, 3, 3, 1, 4, 3, 4, 0, 2, 4, 2, 4, 3, 2,

In [111]:
print("Accuracy: ",(metrics.accuracy_score(Y_test, Predicted_class)*100).round(2))
print("Confusion Matrix: ")
print(metrics.confusion_matrix(Y_test, Predicted_class))
print("Classification Report: ")
print(metrics.classification_report(Y_test,Predicted_class))

Accuracy:  29.1
Confusion Matrix: 
[[20 15  2  7  7]
 [24 33 14 13 18]
 [ 7  9 16  8 15]
 [13 11  8 14 16]
 [45 32 29 31 50]]
Classification Report: 
              precision    recall  f1-score   support

           0       0.18      0.39      0.25        51
           1       0.33      0.32      0.33       102
           2       0.23      0.29      0.26        55
           3       0.19      0.23      0.21        62
           4       0.47      0.27      0.34       187

    accuracy                           0.29       457
   macro avg       0.28      0.30      0.28       457
weighted avg       0.34      0.29      0.30       457



In [112]:
tree.plot_tree(clf)

[Text(0.6456093764081651, 0.9736842105263158, 'x[5] <= 6.5\ngini = 0.798\nsamples = 567\nvalue = [108, 125, 103, 104, 127]'),
 Text(0.3433997189302451, 0.9210526315789473, 'x[6] <= 3.5\ngini = 0.798\nsamples = 487\nvalue = [85, 99, 95, 91, 117]'),
 Text(0.08597692862292718, 0.868421052631579, 'x[2] <= 3.5\ngini = 0.789\nsamples = 133\nvalue = [17, 33, 32, 29, 22]'),
 Text(0.027036770007209804, 0.8157894736842105, 'x[1] <= 21.0\ngini = 0.704\nsamples = 19\nvalue = [2, 3, 3, 9, 2]'),
 Text(0.01802451333813987, 0.7631578947368421, 'x[2] <= 2.5\ngini = 0.64\nsamples = 15\nvalue = [2, 3, 0, 8, 2]'),
 Text(0.008651766402307137, 0.7105263157894737, 'x[4] <= 2.5\ngini = 0.625\nsamples = 4\nvalue = [1, 0, 0, 1, 2]'),
 Text(0.005767844268204758, 0.6578947368421053, 'x[2] <= 1.5\ngini = 0.5\nsamples = 2\nvalue = [1, 0, 0, 1, 0]'),
 Text(0.002883922134102379, 0.6052631578947368, 'gini = 0.0\nsamples = 1\nvalue = [1, 0, 0, 0, 0]'),
 Text(0.008651766402307137, 0.6052631578947368, 'gini = 0.0\nsample

## **Support Vector Machines**

In [113]:
clf = svm.SVC()
clf.fit(X,Y)
Predicted_class = clf.predict(X_test)
Predicted_class

array([1, 1, 1, 4, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 4, 4, 4, 1, 1, 4,
       4, 4, 4, 4, 1, 1, 1, 4, 1, 4, 1, 1, 1, 4, 1, 4, 4, 1, 1, 4, 1, 4,
       1, 1, 1, 1, 1, 1, 1, 4, 4, 1, 1, 4, 4, 1, 4, 4, 4, 4, 1, 4, 4, 1,
       1, 1, 4, 1, 1, 1, 1, 1, 4, 4, 1, 4, 1, 4, 1, 4, 4, 4, 4, 1, 4, 4,
       4, 4, 1, 4, 4, 4, 4, 1, 4, 4, 1, 4, 4, 4, 4, 1, 1, 1, 1, 1, 4, 4,
       1, 1, 4, 4, 1, 1, 1, 1, 4, 1, 1, 4, 1, 4, 1, 4, 4, 1, 4, 4, 1, 1,
       4, 1, 1, 4, 4, 1, 4, 1, 1, 4, 1, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       1, 4, 4, 1, 4, 4, 4, 4, 4, 1, 4, 4, 4, 1, 1, 1, 1, 4, 4, 4, 1, 4,
       4, 4, 4, 4, 4, 4, 4, 1, 4, 4, 4, 4, 1, 4, 4, 1, 4, 4, 4, 1, 4, 1,
       1, 4, 4, 4, 4, 1, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       1, 4, 4, 4, 4, 1, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 1, 4, 1, 4, 4,
       1, 4, 1, 1, 1, 4, 4, 1, 4, 4, 4, 4, 4, 4, 1, 1, 4, 4, 4, 1, 1, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 1, 4, 4, 1, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 1, 1, 4, 4, 1, 4, 4,

In [114]:
print("Accuracy: ",(metrics.accuracy_score(Y_test, Predicted_class)*100).round(2))
print("Confusion Matrix: ")
print(metrics.confusion_matrix(Y_test, Predicted_class))
print("Classification Report: ")
print(metrics.classification_report(Y_test,Predicted_class))

Accuracy:  46.61
Confusion Matrix: 
[[  0  21   0   0  30]
 [  0  64   0   0  38]
 [  0  21   0   0  34]
 [  0  27   0   0  35]
 [  0  38   0   0 149]]
Classification Report: 
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        51
           1       0.37      0.63      0.47       102
           2       0.00      0.00      0.00        55
           3       0.00      0.00      0.00        62
           4       0.52      0.80      0.63       187

    accuracy                           0.47       457
   macro avg       0.18      0.28      0.22       457
weighted avg       0.30      0.47      0.36       457



# **Using Linear SVC**

In [115]:
clf = svm.LinearSVC()
clf.fit(X,Y)
Predicted_class = clf.predict(X_test)
Predicted_class

array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,

In [116]:
print("Accuracy: ",(metrics.accuracy_score(Y_test, Predicted_class)*100).round(2))
print("Confusion Matrix: ")
print(metrics.confusion_matrix(Y_test, Predicted_class))
print("Classification Report: ")
print(metrics.classification_report(Y_test,Predicted_class))

Accuracy:  13.57
Confusion Matrix: 
[[  0   0   0  51   0]
 [  0   0   0 102   0]
 [  0   0   0  55   0]
 [  0   0   0  62   0]
 [  0   0   0 187   0]]
Classification Report: 
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        51
           1       0.00      0.00      0.00       102
           2       0.00      0.00      0.00        55
           3       0.14      1.00      0.24        62
           4       0.00      0.00      0.00       187

    accuracy                           0.14       457
   macro avg       0.03      0.20      0.05       457
weighted avg       0.02      0.14      0.03       457



# **Random Forest**

In [117]:
# dataframe_test['Personality'] = label.fit_transform(dataframe_test['Personality'])
# dataframe_train['Personality'] = label.fit_transform(dataframe_train['Personality'])
X = np.array(dataframe_train[input_columns])
Y = np.array(dataframe_train[output_columns])
X_test = np.array(dataframe_test[input_columns])
Y_test = np.array(dataframe_test[output_columns])
rf = RandomForestRegressor(n_estimators = 10000, random_state = 50, criterion ="poisson")
rf.fit(X,Y)
Predicted_class = rf.predict(X_test)
final = []
for i in Predicted_class:
    final.append(math.ceil(i))
Predicted_class = np.array(final)
Predicted_class

array([3, 1, 3, 3, 3, 4, 2, 3, 2, 3, 2, 2, 2, 3, 1, 2, 3, 3, 3, 1, 3, 3,
       4, 3, 3, 3, 3, 4, 1, 2, 3, 2, 2, 2, 2, 3, 2, 2, 3, 3, 3, 2, 3, 3,
       3, 3, 3, 1, 1, 3, 2, 3, 3, 2, 1, 2, 3, 2, 2, 2, 3, 2, 2, 3, 3, 2,
       3, 4, 2, 3, 2, 4, 2, 2, 3, 3, 1, 2, 3, 3, 2, 3, 3, 2, 3, 2, 2, 3,
       3, 2, 2, 3, 3, 2, 3, 2, 2, 3, 2, 2, 2, 2, 3, 3, 3, 3, 2, 2, 3, 3,
       2, 2, 3, 3, 3, 3, 3, 2, 3, 3, 2, 3, 2, 3, 2, 2, 3, 3, 3, 3, 2, 2,
       3, 2, 2, 3, 3, 2, 3, 3, 2, 3, 3, 2, 3, 2, 3, 3, 2, 2, 3, 3, 2, 3,
       3, 3, 3, 1, 3, 3, 3, 3, 3, 2, 3, 2, 3, 2, 3, 3, 2, 3, 3, 2, 2, 2,
       4, 3, 2, 2, 3, 3, 2, 3, 2, 2, 2, 3, 2, 2, 3, 2, 3, 3, 3, 3, 2, 2,
       2, 3, 3, 3, 2, 2, 2, 3, 2, 3, 3, 3, 3, 2, 3, 2, 2, 4, 3, 2, 2, 2,
       1, 3, 3, 3, 2, 1, 3, 2, 2, 2, 2, 2, 4, 2, 3, 2, 3, 3, 3, 2, 3, 4,
       2, 3, 2, 2, 3, 2, 2, 3, 3, 3, 3, 2, 2, 2, 3, 1, 3, 3, 2, 2, 2, 3,
       3, 2, 4, 2, 3, 2, 2, 2, 2, 2, 3, 3, 2, 3, 3, 2, 3, 2, 3, 3, 3, 3,
       2, 2, 3, 3, 3, 3, 2, 2, 2, 2, 3, 3, 3, 3, 2,

In [118]:
print("Accuracy: ",(metrics.accuracy_score(Y_test, Predicted_class)*100).round(2))
print("Confusion Matrix: ")
print(metrics.confusion_matrix(Y_test, Predicted_class))
print("Classification Report: ")
print(metrics.classification_report(Y_test,Predicted_class))

Accuracy:  15.32
Confusion Matrix: 
[[  0   3  24  21   3]
 [  0   9  54  38   1]
 [  0   0  18  34   3]
 [  0   2  23  36   1]
 [  0   4  75 101   7]]
Classification Report: 
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        51
           1       0.50      0.09      0.15       102
           2       0.09      0.33      0.14        55
           3       0.16      0.58      0.25        62
           4       0.47      0.04      0.07       187

    accuracy                           0.15       457
   macro avg       0.24      0.21      0.12       457
weighted avg       0.33      0.15      0.11       457

